In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import metrics
from sklearn.metrics import accuracy_score



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-data-modified/train_df.csv
/kaggle/input/test-data-modified/test_df.csv


### In this notebook, we try to implement the XGBoost model using Dask-ML. 

In [2]:
# installation of Dask-ML
pip install dask-ml[complete]

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=1) # choosing number of parallel processes
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42331,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:40133,Total threads: 1
Dashboard: http://127.0.0.1:38775/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:36707,


In [4]:
import dask
import xgboost
import dask_xgboost

/opt/conda/lib/python3.7/site-packages/dask_xgboost/__init__.py:8: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  "Dask-XGBoost has been deprecated and is no longer maintained. The functionality "


In [5]:
import dask.dataframe as dd
import dask.array as da


# Loading the Data

In [7]:
df_subset = dd.read_csv('../input/train-data-modified/train_df.csv')

In [8]:
df_subset.head()

,MachineIdentifier,Census_ProcessorCoreCount,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildRevision,...,Census_GenuineStateName_INVALID_LICENSE,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_UNKNOWN,Census_ActivationChannel_OEM:DM,Census_ActivationChannel_OEM:NONSLP,Census_ActivationChannel_Retail,Census_ActivationChannel_Retail:TB:Eval,Census_ActivationChannel_Volume:GVLK,Census_ActivationChannel_Volume:MAK
0,0000028988387b115f69f31a3bf04f09,4.0,13.075148,12.609710,8.318010,2.940,7.273,6.81,22.18071,165,...,0,1,0,0,0,0,1,0,0,0
1,00000b11598a75ea8ba1beea8459149f,4.0,12.382024,12.333220,8.318010,2.918,7.223,6.65,22.18071,228,...,0,1,0,0,0,1,0,0,0,0
2,000014a5f00daa18e76b81417eeb99fc,4.0,13.075148,11.531757,8.723394,2.639,7.223,6.65,0.00000,191,...,0,1,0,0,0,0,1,0,0,0
3,000016191b897145d069102325cab760,2.0,11.648103,11.641072,9.011035,3.068,7.562,6.99,22.18071,165,...,0,1,0,0,0,0,1,0,0,0
4,0000161e8abf8d8b89c5ab8787fd712b,2.0,13.075148,13.036159,8.318010,2.846,7.380,6.81,0.00000,165,...,0,1,0,0,0,0,1,0,0,0


In [9]:
test_df = dd.read_csv('../input/test-data-modified/test_df.csv')

In [10]:
test_df.head()

,MachineIdentifier,Census_ProcessorCoreCount,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildRevision,...,Census_GenuineStateName_INVALID_LICENSE,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_UNKNOWN,Census_ActivationChannel_OEM:DM,Census_ActivationChannel_OEM:NONSLP,Census_ActivationChannel_Retail,Census_ActivationChannel_Retail:TB:Eval,Census_ActivationChannel_Volume:GVLK,Census_ActivationChannel_Volume:MAK
0,000007535c3f730efa9ea0b7ef1bd645,4.0,13.075148,11.536506,8.318010,2.630,7.223,6.65,0.693147,1,...,0,0,1,0,0,0,1,0,0,0
1,000007905a28d863f6d0d597892cd692,4.0,11.648103,11.643147,8.318010,3.068,7.562,6.99,22.180710,165,...,0,1,0,0,0,1,0,0,0,0
2,00001a18d69bb60bda9779408dcf02ac,4.0,13.768283,12.222207,9.011035,2.740,7.223,6.65,0.000000,431,...,0,1,0,0,1,0,0,0,0,0
3,00001b924fcc6922321cfadbafd8a91a,2.0,11.712637,11.702124,8.318010,2.630,7.223,6.65,2.079442,228,...,0,1,0,0,1,0,0,0,0,0
4,000028150912f45b1dc667164de489ef,2.0,12.628873,12.628536,8.318010,2.838,7.273,6.81,0.000000,164,...,0,1,0,0,0,0,1,0,0,0


##### Converting the dataframes to a Dask array. 

In [11]:
main_X = df_subset.drop(['HasDetections', 'MachineIdentifier'], axis=1).to_dask_array(lengths=True)
main_y = df_subset['HasDetections'].to_dask_array(lengths=True)

In [12]:
test_X = test_df.drop(['MachineIdentifier'], axis=1).to_dask_array(lengths=True)

# Scaling

In [13]:
#SCALING NOT WORKING
# from dask_ml.preprocessing import StandardScaler
# scalar = StandardScaler()
# scalar.fit_transform(main_X)

In [14]:
import dask_ml.model_selection as dcv

In [15]:
from dask_ml.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(main_X, main_y, test_size = 0.3, random_state = 42)

In [17]:
# X_train

In [18]:
# X_test

# SGD classifier

In [19]:
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

In [24]:
from dask_ml.xgboost import XGBClassifier

##### Choosing the best hyperparameters, obtained from fine-tuning. 

In [ ]:
params = {'objective': 'binary:logistic',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5}

clf = XGBClassifier()
clf.fit(main_X, main_y)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


In [ ]:
y_pred = clf.predict(X_test)
print('Accuracy Score: ', (accuracy_score(y_test, y_pred)))
print('ROC-AUC score: ', metrics.roc_auc_score(y_test, y_pred))

In [ ]:
#kaggle_test_X = test_df.drop(columns="MachineIdentifier")
kaggle_test_yhat = clf.predict_proba(test_X)
#kaggle_test_yhat = clf.predict_proba(kaggle_test_X)
#kaggle_test_yhat = clf.predict_proba(kaggle_test_X)[:, -1] # predicting the probabilities
test_df["HasDetections"] = kaggle_test_yhat
to_submit = test_df[["MachineIdentifier", "HasDetections"]]
to_submit.to_csv("class-sgd.csv", single_file=True, index=False)

In [ ]:
sorted(clf.cv_results_.keys())